## Cardinalidad

Los valores de las variables categóricas son seleccionados de un grupo de categorías, también conocidos como etiquetas. Por ejemplo, en la variable _género_ las categorías o etiquetas son masculino y femenino, mientras que la variable  _ciudad_ las etiquetas pueden ser Londres, Manchester, Brighton y así sucesivamente.

Diferentes variables categóricas tienen diferentes números de categorías. La variable género tiene dos etiquetas, pero la variable ciudad o código postal, pueden tener un gran número de categorías.  

Al número de etiquetas diferentes en una variable categórica se le conoce como cardinalidad. Cuando el número de etiquetas es alto, se dice que la variable categórica tiene __alta cardinalidad__.


### Son múltiple etiquetas en una variable categórica un problema?

La alta cardinalidad puede ser problemática tanto para estadísticas descriptivas y como para modelamiento predictivo:

- Variables con demasiadas etiquetas pueden dominar sobre otras con baja cardinalidad, particularmente en algoritmos basados en **árboles de decisión**. Estos algoritmos le dan más importancia a variables de alta cardinalidad ya que suelen aumentar de la ganancia de la información cuando se están ajustando. Pero luego, estos árboles terminan sobre-ajustándose al set de entrenamiento, y muestran una mala generalización.

- Un gran número de etiquetas en las variables puede introducir ruido, por lo tanto, modelos de 'machine learning' tienden a sobre-ajustar. 

- Algunas de las etiquetas pueden estar presentes en los datos usados para entrenar el modelo predictivo, pero no los datos usados para pruebas, por lo tanto el modelo puede sobre-ajustar a los datos de entrenamiento.

- Por el contrario, algunas de las etiquetas pueden aparecer solamente en los datos de prueba, por lo tanto dejando el modelo predictivo incapaz de realizar los cálculos en las nuevas categorías no antes vistas.

En particular,  **algoritmos basados en árboles de decisión suelen estar sesgados hacia variables con un gran número de etiquetas **. Por lo tanto su desempeño se afecta por alta cardinalidad.

A continuación, demostraremos el efecto de alta cardinalidad en el desempeño de diferentes algoritmos de machine learning y como se puede solucionar este problema, simplemente con reducir el número de etiquetas.



## En este Demo:

En las siguientes celdas vamos a:

- Aprender como cuantificar cardinalidad
- Ver ejemplos de variables con alta y baja cardinalidad
- Entender el efecto de cardinalidad cuando preparamos sets de datos para entrenamiento y pruebas de un modelo predictivo
- Visualizar el efecto de cardinalidad en el desempeño de modelos de Machine Learning.

En este demo, usaremos los datos del **Titanic**

- Para descargar los datos, por favor referirse a la sección de **Datos** de la **Sección 1** del curso.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# para construir modelos de machine learning 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# para evaluar los modelos
from sklearn.metrics import roc_auc_score

# para separar datos en sets de entrenamiento y prueba
from sklearn.model_selection import train_test_split

In [2]:
# carguemos los datos del titanic 

data = pd.read_csv('../titanic.csv')
data.head()

pclass  survived                                             name     sex  \
0       1         1                    Allen, Miss. Elisabeth Walton  female   
1       1         1                   Allison, Master. Hudson Trevor    male   
2       1         0                     Allison, Miss. Helen Loraine  female   
3       1         0             Allison, Mr. Hudson Joshua Creighton    male   
4       1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

       age  sibsp  parch  ticket      fare cabin embarked boat   body  \
0  29.0000      0      0   24160  211.3375    B5        S    2    NaN   
1   0.9167      1      2  113781  151.5500   C22        S   11    NaN   
2   2.0000      1      2  113781  151.5500   C22        S  NaN    NaN   
3  30.0000      1      2  113781  151.5500   C22        S  NaN  135.0   
4  25.0000      1      2  113781  151.5500   C22        S  NaN    NaN   

                         home.dest  
0                     St Louis, MO  
1  Montreal, PQ / Chesterville, ON  
2  Montreal, PQ / Chesterville, ON  
3  Montreal, PQ / Chesterville, ON  
4  Montreal, PQ / Chesterville, ON

Las variables categóricas en estos datos son Name (nombre), Sex (género), Ticket ( Tiquete), Cabin (cabina) y Embarked (puerto de embarque).

---------------
**Nota** Ticket y Cabin son variables alfanúmericas, es decir contienen letras y números, asi q deben ser tratados como Variables Combinadas. Para este demo, las manipularemos como categóricas.


In [3]:
# Miremos la cardinalidad para cada una de las variables

# Es decir, el número de diferentes categorías o etiquetas.

print('Número de diferentes categorías en la variable Name: {}'.format(
    len(data.name.unique())))

print('Número de diferentes categorías en la variable Gender: {}'.format(
    len(data.sex.unique())))

print('Número de diferentes categorías en la variable Ticket: {}'.format(
    len(data.ticket.unique())))

print('Número de diferentes categorías en la variable Cabin: {}'.format(
    len(data.cabin.unique())))

print('Número de diferentes categorías en la variable Embarked: {}'.format(
    len(data.embarked.unique())))

print('Número total de pasajeros en el Titanic: {}'.format(len(data)))

Número de diferentes categorías en la variable Name: 1307
Número de diferentes categorías en la variable Gender: 2
Número de diferentes categorías en la variable Ticket: 929
Número de diferentes categorías en la variable Cabin: 182
Número de diferentes categorías en la variable Embarked: 4
Número total de pasajeros en el Titanic: 1309


Mientras que la variable Sex tiene solo 2 etiquetas y Embarked 4 (baja cardinalidad), las variables Ticket, Name y Cabin, como es de esperarse, contienen un gran número de etiquetas (alta cardinalidad).

Para demostrar, el efecto de alta cardinalidad en la preparación de datos para entrenamiento y pruebas de un modelo de machine learning así como su desempeño usaremos la variable Cabin. 
Crearemos una nueva variable con cardinalidad reducida.


In [4]:
# exploremos los valores / categorías de Cabin

# Podemos observar que hay 182 diferentes cabinas
# por lo tanto la variable tienen alta cardinalidad

data.cabin.unique()

array(['B5', 'C22', 'E12', 'D7', 'A36', 'C101', nan, 'C62', 'B35', 'A23',
       'B58', 'D15', 'C6', 'D35', 'C148', 'C97', 'B49', 'C99', 'C52', 'T',
       'A31', 'C7', 'C103', 'D22', 'E33', 'A21', 'B10', 'B4', 'E40',
       'B38', 'E24', 'B51', 'B96', 'C46', 'E31', 'E8', 'B61', 'B77', 'A9',
       'C89', 'A14', 'E58', 'E49', 'E52', 'E45', 'B22', 'B26', 'C85',
       'E17', 'B71', 'B20', 'A34', 'C86', 'A16', 'A20', 'A18', 'C54',
       'C45', 'D20', 'A29', 'C95', 'E25', 'C111', 'C23', 'E36', 'D34',
       'D40', 'B39', 'B41', 'B102', 'C123', 'E63', 'C130', 'B86', 'C92',
       'A5', 'C51', 'B42', 'C91', 'C125', 'D10', 'B82', 'E50', 'D33',
       'C83', 'B94', 'D49', 'D45', 'B69', 'B11', 'E46', 'C39', 'B18',
       'D11', 'C93', 'B28', 'C49', 'B52', 'E60', 'C132', 'B37', 'D21',
       'D19', 'C124', 'D17', 'B101', 'D28', 'D6', 'D9', 'B80', 'C106',
       'B79', 'C47', 'D30', 'C90', 'E38', 'C78', 'C30', 'C118', 'D36',
       'D48', 'D47', 'C105', 'B36', 'B30', 'D43', 'B24', 'C2', 'C65',


Ahora reduzcamos la cardinalidad de esta variable. 
Cómo? en vez de usar toda la variable Cabin, capturaremos solamente la primera letra.

***Justificación***: la primera letra indica el piso en el cual la cabina se encontraba, y por lo tanto, es un indicativo de la clase social y estatus, así como de la proximidad a la superficie del Titanic. Ambos se conocen por mejorar la probabilidad de sobrevivir en el Titanic.


In [5]:
# let's capture the first letter of Cabin

data['Cabin_reduced'] = data['cabin'].astype(str).str[0]

data[['cabin', 'Cabin_reduced']].head()

cabin Cabin_reduced
0    B5             B
1   C22             C
2   C22             C
3   C22             C
4   C22             C

In [6]:
print('Número de diferentes categorías en la variable Cabin: {}'.format(
    len(data.cabin.unique())))

print('Número de diferentes categorías en la variable Cabin reducida: {}'.format(
    len(data.Cabin_reduced.unique())))

Número de diferentes categorías en la variable Cabin: 182
Número de diferentes categorías en la variable Cabin reducida: 9


Se redujeron el número de diferentes etiquetas de 182 a 9.

In [7]:
# separemos los datos para entrenamiento y pruebas
# para constuir modelo de machine learning

use_cols = ['cabin', 'Cabin_reduced', 'sex']

# tomemos función de scikit-learn
X_train, X_test, y_train, y_test = train_test_split(
    data[use_cols], 
    data.survived,  
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape


((916, 3), (393, 3))

### Alta cardinalidad lleva a una distribución desigual de las categorías para las muestras usadas para entrenamiento y pruebas.

Cuando una variable es altamente cardinal, frecuentemente las categorías solo aparecen en la muestra de datos para entrenamiento, lo cual tiende a generar sobreajustes.

Por el contrario, si las categorías están presentes solo en la muestra de pruebas (o nuevos datos cuando el modelo se encuentra en producción), el algoritmo no sabrá cómo manejar estas nuevas etiquetas. 

Este problema se resuelve en gran parte reduciendo la cardinalidad de la variable. Veamos a continuación:

In [8]:
# Busquemos las etiquetas presentes solamente en el conjunto de datos para entrenamiento 

unique_to_train_set = [
    x for x in X_train.cabin.unique() if x not in X_test.cabin.unique()
]

len(unique_to_train_set)

113

Hay 113 cabinas solamente presentes en la muestra de entrenamiento y no en el de pruebas.

In [9]:
# Busquemos las etiquetas presentes solamente en el conjunto de datos para pruebas 

unique_to_test_set = [
    x for x in X_test.cabin.unique() if x not in X_train.cabin.unique()
]

len(unique_to_test_set)

36

Vemos como para una variable con alta cardinalidad,  valores (categorías) presentes en el conjunto de datos para entrenamiento no están en la muestra de prueba y viceversa. 

Ahora, este problema se puede resolver, reduciendo la cardinalidad de la variable como se muestra a continuación:

In [10]:
# Veamos las etiquetas presentes en los datos de entrenamiento
# para la variable Cabin cardinalidad reducida

unique_to_train_set = [
    x for x in X_train['Cabin_reduced'].unique()
    if x not in X_test['Cabin_reduced'].unique()
]

len(unique_to_train_set)

1

In [11]:
# Veamos las etiquetas presentes en los datos de prueba
# para la variable Cabin cardinalidad reducida

unique_to_test_set = [
    x for x in X_test['Cabin_reduced'].unique()
    if x not in X_train['Cabin_reduced'].unique()
]

len(unique_to_test_set)

0

Podemos ver como reduciendo la cardinalidad conlleva a solo tener una categoría en los datos de entrenamiento que no esta presente en los de prueba.  Y todas las etiquetes en el conjunto de prueba estan enel de entrenamiento.

### Efecto de la cardinalidad en el desempeño de los modelos de Machine Learning

Para evaluar el efecto de las variables categóricas en los modelos de machine learning, reemplacemos las categorías por números.

In [12]:
# Reemplacemos cada cabina por un número
# para demostrar rápidamente el efecto de las
# etiquetas en los algoritmos de machine learning

##############
# Nota: esta no es ni la única ni la mejor forma de  
# codificar variables categóricas en números 
# Veremos más detalles sobre estas y otras técnicas de codificación
# de variables categóricas en la sección respectiva: 
# "Codificando variables categóricas"
##############


cabin_dict = {k: i for i, k in enumerate(X_train.cabin.unique(), 0)}
cabin_dict

{nan: 0,
 'E36': 1,
 'C68': 2,
 'E24': 3,
 'C22': 4,
 'D38': 5,
 'B50': 6,
 'A24': 7,
 'C111': 8,
 'F': 9,
 'C6': 10,
 'C87': 11,
 'E8': 12,
 'B45': 13,
 'C93': 14,
 'D28': 15,
 'D36': 16,
 'C125': 17,
 'B35': 18,
 'T': 19,
 'B73': 20,
 'B57': 21,
 'A26': 22,
 'A18': 23,
 'B96': 24,
 'G6': 25,
 'C78': 26,
 'C101': 27,
 'D9': 28,
 'D33': 29,
 'C128': 30,
 'E50': 31,
 'B26': 32,
 'B69': 33,
 'E121': 34,
 'C123': 35,
 'B94': 36,
 'A34': 37,
 'D': 38,
 'C39': 39,
 'D43': 40,
 'E31': 41,
 'B5': 42,
 'D17': 43,
 'F33': 44,
 'E44': 45,
 'D7': 46,
 'A21': 47,
 'D34': 48,
 'A29': 49,
 'D35': 50,
 'A11': 51,
 'B51': 52,
 'D46': 53,
 'E60': 54,
 'C30': 55,
 'D26': 56,
 'E68': 57,
 'A9': 58,
 'B71': 59,
 'D37': 60,
 'F2': 61,
 'C55': 62,
 'C89': 63,
 'C124': 64,
 'C23': 65,
 'C126': 66,
 'E49': 67,
 'E46': 68,
 'D19': 69,
 'B58': 70,
 'C82': 71,
 'B52': 72,
 'C92': 73,
 'E45': 74,
 'C65': 75,
 'E25': 76,
 'B3': 77,
 'D40': 78,
 'C91': 79,
 'B102': 80,
 'B61': 81,
 'A20': 82,
 'B36': 83,
 'C7': 84,

In [13]:
# codificar la variable cabina utilizando el diccionario creado anteriormente
# notemos que el diccionario reemplaza valores nulos por zero

X_train.loc[:, 'Cabin_mapped'] = X_train.loc[:, 'cabin'].map(cabin_dict)
X_test.loc[:, 'Cabin_mapped'] = X_test.loc[:, 'cabin'].map(cabin_dict)

X_train[['Cabin_mapped', 'cabin']].head(10)

Cabin_mapped cabin
501              0   NaN
588              0   NaN
402              0   NaN
1193             0   NaN
686              0   NaN
971              0   NaN
117              1   E36
540              0   NaN
294              2   C68
261              3   E24

Vemos como NaN toma el valor de 0 en la nueva variable, E36 toma el valor de 1, C68 toma el valor de 2 y asi sucesivamente.

In [14]:
# Ahora reemplazamos las letras en la variable
# cabina reducida siguiendo el mismo procedimiento

# crear diccionario para reeemplazar
cabin_dict = {k: i for i, k in enumerate(X_train['Cabin_reduced'].unique(), 0)}

# reemplazar las etiquetas por los números con el diccionario
X_train.loc[:, 'Cabin_reduced'] = X_train.loc[:, 'Cabin_reduced'].map(
    cabin_dict)
X_test.loc[:, 'Cabin_reduced'] = X_test.loc[:, 'Cabin_reduced'].map(cabin_dict)

X_train[['Cabin_reduced', 'cabin']].head(10)

Cabin_reduced cabin
501               0   NaN
588               0   NaN
402               0   NaN
1193              0   NaN
686               0   NaN
971               0   NaN
117               1   E36
540               0   NaN
294               2   C68
261               1   E24

Vemos que E36 y E24 tiene el mismo número, 1, porque estamos capturando solo la información de letras y ambos empiezan con E.

In [15]:
# Ahora reemplacemos las categorías de la variable Sex con números

X_train.loc[:, 'sex'] = X_train.loc[:, 'sex'].map({'male': 0, 'female': 1})
X_test.loc[:, 'sex'] = X_test.loc[:, 'sex'].map({'male': 0, 'female': 1})

X_train.sex.head()

501     1
588     1
402     1
1193    0
686     1
Name: sex, dtype: int64

In [16]:
# miremos si hay datos faltantes en estas variables

X_train[['Cabin_mapped', 'Cabin_reduced', 'sex']].isnull().sum()

Cabin_mapped     0
Cabin_reduced    0
sex              0
dtype: int64

In [17]:
X_test[['Cabin_mapped', 'Cabin_reduced', 'sex']].isnull().sum()

Cabin_mapped     41
Cabin_reduced     0
sex               0
dtype: int64

En el set de prueba, hay ahora 41 valores faltantes para la variable de alta cardinalidad. Estas fueron introducidas cuando re-asignamos las categorías a números. Cómo sucedió? muchas categorías existen solo en la muestra de prueba. Inicialmente creamos nuestro  diccionario  usando la muestra de entrenamiento, y no generamos números para reemplazar esas etiquetas que se encontraban solo en la muestra de prueba. El resultado, es que estas etiquetas se codificaron con NaN. Veremos en futuros Notebooks cómo manejar este problema. Por ahora, llenaremos esos datos faltantes con 0.


In [18]:
# Miremos el número de las diferentes categorias en las variables codificadas.
len(X_train.Cabin_mapped.unique()), len(X_train.Cabin_reduced.unique())

(147, 9)

Podemos ver que de las 182 cabinas originales en los datos, solo 147 están presentes en la muestra de entrenamiento. Vemos también como reducimos el número de las diferentes categorías a solo 9.

Prosigamos con la evaluación del efecto de la cardinalidad en los algoritmos de machine learning.


### Random Forests

In [19]:
# Modelo construido con los datos de alta cardinalidad 
# en la variable cabina

# Modelo
rf = RandomForestClassifier(n_estimators=200, random_state=39)

# entrenar el modelo
rf.fit(X_train[['Cabin_mapped', 'sex']], y_train)

# hacer las predicciones en la muestra de entrenamiento y prueba
pred_train = rf.predict_proba(X_train[['Cabin_mapped', 'sex']])
pred_test = rf.predict_proba(X_test[['Cabin_mapped', 'sex']].fillna(0))

print('Muestra de entrenamiento')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Muestra de prueba')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Muestra de entrenamiento
Random Forests roc-auc: 0.853790650048556
Muestra de prueba
Random Forests roc-auc: 0.7691361097284443


Podemos ver que el desempeño de Random Forest en la muestra de entrenamiento es muy superior al desempeño en la muestra de prueba. Esto indica que el modelo esta sobre-ajustando: es muy bueno prediciendo los resultados en los datos que se usan para entrenarlo, pero no tiene el poder para generalizar las predicciones en datos nuevos.


In [20]:
# modelo construido con dataos de baja cardinalidad para cabin

# iniciar modelo
rf = RandomForestClassifier(n_estimators=200, random_state=39)

# entrenar  modelo
rf.fit(X_train[['Cabin_reduced', 'sex']], y_train)

# hacer predicciones en el segmento de prueba y entrenamiento 
pred_train = rf.predict_proba(X_train[['Cabin_reduced', 'sex']])
pred_test = rf.predict_proba(X_test[['Cabin_reduced', 'sex']])

print('Segmento de entrenamiento')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Segmento de prueba')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Segmento de entrenamiento
Random Forests roc-auc: 0.8163420365403872
Segmento de prueba
Random Forests roc-auc: 0.8017670482827277


Podemos ver que con variables de baja cardinalidad, el Random Forest no sobre-ajusta al segmento de entrenamiento. Además, el modelo es mucho mejor generalizando las predicciones (compara el roc-auc de este modelo en el segmento de prueba vs el roc-auc del modelo anterior).

**Algo para resaltar: probablemente se pueda mejorar el impacto de la alta cardinalidad ajustando los hiper-parámetros del Random Forest. Esto sin embargo, va más allá del alcance de este curso. Aquí simplemente queremos mostrar, que dado el mismo modelo, con los mismos hiper-parámetros, la alta cardinalidad puede causar sobre-ajustes en el mismo.**.


### AdaBoost

In [21]:
# modelo construido con datos de alta cardinalidad para cabin

# iniciar modelo
ada = AdaBoostClassifier(n_estimators=200, random_state=44)

# entrenar  modelo
ada.fit(X_train[['Cabin_mapped', 'sex']], y_train)

# hacer predicciones en el segmento de prueba y entrenamiento 
pred_train = ada.predict_proba(X_train[['Cabin_mapped', 'sex']])
pred_test = ada.predict_proba(X_test[['Cabin_mapped', 'sex']].fillna(0))

print('Segmento de entrenamiento')
print('Adaboost roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Segmento de prueba')
print('Adaboost roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Segmento de entrenamiento
Adaboost roc-auc: 0.8296861713101102
Segmento de prueba
Adaboost roc-auc: 0.7604391350035948


In [22]:
# modelo construido con datos de baja cardinalidad para cabin

# iniciar modelo
ada = AdaBoostClassifier(n_estimators=200, random_state=44)

# entrenar  modelo
ada.fit(X_train[['Cabin_reduced', 'sex']], y_train)

#  predicciones en el segmento de prueba y entrenamiento 
pred_train = ada.predict_proba(X_train[['Cabin_reduced', 'sex']])
pred_test = ada.predict_proba(X_test[['Cabin_reduced', 'sex']].fillna(0))

print('Segmento de entrenamiento')
print('Adaboost roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Segmento de prueba')
print('Adaboost roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))


Segmento de entrenamiento
Adaboost roc-auc: 0.8161256723642566
Segmento de prueba
Adaboost roc-auc: 0.8001078480172557


Similarmente, el modelo Adaboost entrenado con la variable con alta cardinalidad sobre-ajusta el segmento de entrenamiento. Por el contrario, el modelo Adaboots entrenado con la variable de baja cardinalidad, generaliza mejor las predicciones.

Adicionalmente hay otras ventajas en construir un modelo con menos categorías en Cabina: a) es más simple y b) si una nueva categoría aparece en el segmento de prueba, simplemente con tomar la primera letra de la cabina, el modelo de machine learning sabrá cómo manejarla ya que ese caso “existía” en los datos de entrenamiento.


### Regresión Logística 

In [23]:
# modelo construido con datos de alta cardinalidad para cabin

# iniciar modelo
logit = LogisticRegression(random_state=44, solver='lbfgs')

# entrenar modelo
logit.fit(X_train[['Cabin_mapped', 'sex']], y_train)

#  hacer predicciones en el segmento de prueba y entrenamiento 
pred_train = logit.predict_proba(X_train[['Cabin_mapped', 'sex']])
pred_test = logit.predict_proba(X_test[['Cabin_mapped', 'sex']].fillna(0))

print('Segmento de entrenamiento')
print('Logistic regression roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Segmento de prueba')
print('Logistic regression roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Segmento de entrenamiento
Logistic regression roc-auc: 0.8133909298124677
Segmento de prueba
Logistic regression roc-auc: 0.7750815773463858


In [24]:
# modelo construido con datos de baja cardinalidad para cabin

# iniciar modelo
logit = LogisticRegression(random_state=44, solver='lbfgs')

# entrenar modelo
logit.fit(X_train[['Cabin_reduced', 'sex']], y_train)

#  hacer predicciones en el segmento de prueba y entrenamiento 
pred_train = logit.predict_proba(X_train[['Cabin_reduced', 'sex']])
pred_test = logit.predict_proba(X_test[['Cabin_reduced', 'sex']].fillna(0))

print('Segmento de entrenamiento')
print('Logistic regression roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Segmento de prueba')
print('Logistic regression roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Segmento de entrenamiento
Logistic regression roc-auc: 0.8123468468695123
Segmento de prueba
Logistic regression roc-auc: 0.8008268347989602


Podemos sacar la misma conclusión con la Regresión Logistica: reducir la cardinalidad mejora el desempeño del modelo y el poder de generalización del mismo


### Gradient Boosted Classifier - Máquina de potenciación del gradiente

In [25]:
# Construir modelo con datos con variedad categorias
# en la variable cabin

# iniciar modelo
gbc = GradientBoostingClassifier(n_estimators=300, random_state=44)

# entrenar modelo
gbc.fit(X_train[['Cabin_mapped', 'sex']], y_train)

#  hacer predicciones en el segmento de prueba y entrenamiento 
pred_train = gbc.predict_proba(X_train[['Cabin_mapped', 'sex']])
pred_test = gbc.predict_proba(X_test[['Cabin_mapped', 'sex']].fillna(0))

print('Segmento de entrenamiento')
print('Máquina de potenciación del gradiente roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Segmento de prueba')
print('Máquina de potenciación del gradiente roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))



Segmento de entrenamiento
Máquina de potenciación del gradiente roc-auc: 0.862631390919749
Segmento de prueba
Máquina de potenciación del gradiente roc-auc: 0.7733117637298823


In [26]:
# Construir modelo con datos con variedad categorias
# en la variable cabin

# iniciar modelo
gbc = GradientBoostingClassifier(n_estimators=300, random_state=44)

# entrenar modelo
gbc.fit(X_train[['Cabin_reduced', 'sex']], y_train)

#  hacer predicciones en el segmento de prueba y entrenamiento 
pred_train = gbc.predict_proba(X_train[['Cabin_reduced', 'sex']])
pred_test = gbc.predict_proba(X_test[['Cabin_reduced', 'sex']].fillna(0))


print('Segmento de entrenamiento')
print('Máquina de potenciación del gradiente roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Segmento de prueba')
print('Máquina de potenciación del gradiente roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Segmento de entrenamiento
Máquina de potenciación del gradiente roc-auc: 0.816719415917359
Segmento de prueba
Máquina de potenciación del gradiente roc-auc: 0.8015181682429069


Gradient Boosted trees sobre ajustan al segmento de entrenamiento en los casos donde la variable cabin tiene muchas etiquetas. Esto es de esperarse ya que los métodos basados en arboles tienden a sesgarse cuando hay muchas categorías en una variable.

**Esto es todo por este demo, esperamos lo hayan disfrutado y nos vemos en el siguiente. **